In [1]:
import re
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
# !ls

In [4]:
fname = 'avito_2022-08-26_17-29_raw.xlsx'
df = pd.read_excel('data/'+fname)
print(len(df))
df.sample(5)

3949


,avito_id,title,price,adr,description
3198,2437168243,"Объявление «Квартира-студия, 25 м², 2/3 эт.» 4 фотографии",3900000,"ул. Генерала Лебедя, 6Б","Документы готовы. Дом на кадастровом учёте стоит в росреестре с присвоенным адресом. Зарегистрирован, как жилой. Евро ремонт, кухня встроенная на лоджии, мойка-природный камень. В ванной плитка, новая сантехника, стиральная машина. Натяжные потолки. Тихий"
2459,2281405453,"Объявление «Квартира-студия, 30 м², 6/8 эт.» 14 фотографий",6390000,"ул. Токарева, 18В","Продаётся новая студия, единственная откуда видно море, с отличным ремонтом, в одном из лучших жилищных комплексов г. Севастополь ЖК «Доброгород» — самый современный семейный микрорайон с домами выполненными по специально разработанному дизайну и цветовой"
2037,2473430162,"Объявление «1-к. квартира, 30,7 м², 1/5 эт.» 17 фотографий",4995000,"ул. Глухова, 5","В продаже однокомнатная квартира в Стрелецкой бухте. В квартире выполнен хороший косметический ремонт, вся мебель и техника (кроме стеклянного стола на кухне) остается новому владельцу. Горячая вода от бойлера, отопление центральное, плита газовая. Квартир"
790,2504969343,"Объявление «1-к. квартира, 34 м², 7/9 эт.» с фотографией",5950000,"ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 1","Крупнейший в Севастополе семейный микрорайон «Доброгород»! \n\nПобедитель всероссийской премии «Топ ЖК 2021» в номинации «Лучший жилой комплекс». \n\nИпотека с господдержкой, рассрочка от Застройщика. \n\n«Доброгород» по праву считается лучшим жилым кварталом Се"
3936,2430928969,"Объявление «1-к. квартира, 43,3 м², 1/10 эт.» 9 фотографий",6300000,"ул. Горпищенко, 127к5",Однокомнатная квартира в новом сданном доме с документами! \n\nУдобная правильная планировка пространства. Автономное газовое отопление. Приборы учётов. Состояние — от строителей. \n\nОбщая площадь 43.3 кв. М. \n\nКомната — 20 кв. М. \n\nКухня — 12 кв. М. \n\nБалкон


In [5]:
df['title'] = df['title'].str.lower().str.extract( r'.*«(.*)».*',expand=False)

In [6]:
df['nrooms'] = df['title'].str.extract( r'.*(\d)-к. .*',expand=False)
df['floor'] = df['title'].str.extract( r'.*(\d+)/\d+.эт.*',expand=False)
df['nfloors'] = df['title'].str.extract( r'.*\d+/(\d+).эт.*',expand=False)
df['area'] = (
    df['title']
    .str.extract( r'.*, ([\d,]+).*м²,.*',expand=False)
    .str.replace(',','.')
)

df['is_studio'] = df['title'].str.lower().str.match(r'.*студи.*')
df['is_apartment'] = df['title'].str.lower().str.match(r'.*апартамент.*')
df['is_part'] = df['title'].str.lower().str.match(r'.*дол.*')
df['is_auction'] = df['title'].str.lower().str.match(r'.*аукци.*')
df['is_openspace'] = df['title'].str.lower().str.match(r'.*своб.*планир.*')

df['is_roof'] = df['description'].str.lower().str.match('.*мансард.*')

df['is_SNT'] = (
    df['adr'].str.match('.*СТ .*') 
    | df['adr'].str.match('.*СНТ .*') 
    | df['adr'].str.lower().str.match('.*садов.*')
)

df['is_fiolent'] = (
    df['description'].str.lower().str.match('.*фиолент.*')
    | df['adr'].str.lower().str.match('.*фиолент.*')
)
    
# df[ df['adr'].str.match('.*СТ .*') ]
# df[ df['adr'].str.match('.*СНТ .*') ]
# df[ df['adr'].str.match('.*садов.*') ]
# df[ df['description'].str.match('.*садов.*') ]

In [7]:
# df['avito_id'] = df['avito_id'].astype(int)
df['price'] = df['price'].astype(int)
df['nrooms'] = df['nrooms'].fillna('0').astype(int)
df['floor'] = df['floor'].fillna('0').astype(int)
df['nfloors'] = df['nfloors'].fillna('0').astype(int)
df['area'] = df['area'].fillna('0.').astype(float)
df['priceM'] = df['price']/1e6

In [8]:
df['is_last_floor'] = ( df['floor'] == df['nfloors'] )

In [9]:
df['adr']

0                              ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 3
1                              ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 1
2                                                        ул. Генерала Жидилова, 4
3                                                          ул. Николая Музыки, 96
4                                      Апарт-комплекс «GARDEN»ул. Лётчиков, д. 10
                                          ...                                    
3944    Нахимовский муниципальный округ, посёлок Любимовка, Южногородская ул., 36
3945                                                        ул. Загордянского, 24
3946                                                      ул. Генерала Лебедя, 33
3947                                                      ул. Генерала Лебедя, 18
3948                                                            Античный пр-т, 24
Name: adr, Length: 3949, dtype: object

In [9]:
cols = [
 'adr',
 'title',
 'priceM',
 'nrooms',
 'floor',
 'nfloors',
 'area',
 'is_studio',
 'is_apartment',
 'is_part',
 'is_auction',
 'is_openspace',
 'is_SNT',
 'is_fiolent',   
 'is_last_floor', 
 'is_roof',   
 'description',
 'price',
 'avito_id',
]

fname = fname.replace('_raw','')

# df[cols].to_pickle(fname+'.pkl',index=False)
df[cols].to_excel('data/'+fname,index=False)

In [10]:
df[cols].sample(10)

,adr,title,priceM,nrooms,floor,nfloors,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_SNT,is_fiolent,is_last_floor,is_roof,description,price,avito_id
1730,"ЖК «Новый»ш. Лабораторное, д. 33, секц. 4","1-к. квартира, 36,4 м², 4/12 эт.",5.63,1,4,12,36.40,False,False,False,False,False,False,False,False,False,"В продаже однокомнатная квартира на 4 этаже двенадцатиэтажного дома в новом комплексе жилых — ЖК «Новый» (4 корпус), Нахимовский район города Севастополь. \n\nПодробнее о квартире: \n\n— Общая площадь квартиры: 36,39 м²;\n— Комфортная кухня: 11,25 м²;\n— Комната",5633172,2460876373
2926,"ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 3","1-к. квартира, 47,2 м², 5/9 эт.",7.55,1,5,9,47.20,False,False,False,False,False,False,False,False,False,"Крупнейший в Севастополе семейный микрорайон «Доброгород»! \n\nПобедитель всероссийской премии «Топ ЖК 2021» в номинации «Лучший жилой комплекс». \n\nИпотека с господдержкой, рассрочка от Застройщика. \n\n«Доброгород» по праву считается лучшим жилым кварталом Се",7552000,2505042729
2478,"ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 1","1-к. квартира, 38,2 м², 6/9 эт.",6.11,1,6,9,38.20,False,False,False,False,False,False,False,False,False,"Крупнейший в Севастополе семейный микрорайон «Доброгород»! \n\nПобедитель всероссийской премии «Топ ЖК 2021» в номинации «Лучший жилой комплекс». \n\nИпотека с господдержкой, рассрочка от Застройщика. \n\n«Доброгород» по праву считается лучшим жилым кварталом Се",6112000,2505373941
958,"ул. Вакуленчука, 53/7","1-к. квартира, 51 м², 5/5 эт.",6.90,1,5,5,51.00,False,False,False,False,False,False,False,True,False,"Продам крупногабаритную однокомнатную квартиру, общей площадью 51 кв. М с автономным отоплением. Большая просторная комната с выходом на уютную лоджию, кухня 11 кв. М и большая ванная в итальянском стиле. Дизайнерский подход прослеживается в каждом помещен",6900000,2510067749
1602,"пр-т Победы, 31А","1-к. квартира, 35,2 м², 1/5 эт.",5.10,1,1,5,35.20,False,False,False,False,False,False,False,False,False,"Продается 1-к квартира проект «чешка» в Нахимовском районе по адресу: проспект Победы 31А. Квартира расположена на первом высоком этаже (есть цоколь). Общая площадь 35, 2 м2, комната 19,2 м2, кухня 7 м2, просторный коридор с кладовой, совмещенный санузел,",5100000,2475354768
2947,"ЖК «Голубая Бухта»аллея Защитников 35-й Батареи, д. 11а","1-к. квартира, 33 м², 2/3 эт.",3.94,1,2,3,33.00,False,False,False,False,False,False,False,False,False,"ЖК «Голубая Бухта» — это неповторимое сочетание современной архитектуры и комфортной жизни с удобным расположением у самого берега Черного моря и живописным видом на Голубую бухту. \nКомплекс имеет уникальную инфраструктуру:300м до пляжа «Голубая бухта», Sp",3939600,2372746251
1549,"ул. Военных Строителей, 10/7","1-к. квартира, 31,8 м², 3/6 эт.",5.85,1,3,6,31.80,False,False,False,False,False,False,False,False,False,"Продам однокомнатную квартиру в спокойном и красивом месте. Квартира расположена в Казачьей бухте (Гагаринский район). Дом 2013 года постройки. В квартире хороший ремонт, с мебелью и техникой. Все чисто и аккуратно. Стоит котёл Агв. \nТеневая сторона, окна",5850000,2501908859
2403,"ул. Гавена, 8","2-к. квартира, 41 м², 1/2 эт.",6.50,2,1,2,41.00,False,False,False,False,False,False,False,False,False,"Двухкомнатная квартира.\nГазовая колонка, пластиковые окна, новый кафель. \nХорошее состояние. \nЭтаж 1ый очень высокий. \nСвой кусочек земли. \nДве комнаты 12 и 13 метров. \nДо моря 7 минут ходьбы. \nМожно в Ипотеку.",6500000,2441364808
115,"ЖК «Голубая Бухта»аллея Защитников 35-й Батареи, д. 11а","1-к. квартира, 26 м², 2/3 эт.",3.93,1,2,3,26.00,False,False,False,False,False,False,False,False,False,"ЖК «Голубая Бухта» — это неповторимое сочетание современной архитектуры и комфортной жизни с удобным расположением у самого берега Черного моря и живописным видом на Голубую бухту. \nКомплекс имеет уникальную инфраструктуру:300м до пляжа «Голубая бухта», Sp",3928513,2372727161
832,"ЖК «Корабе

In [11]:
# df.query('is_SNT')

In [12]:
# df['avito_id'] = df['avito_id'].astype(int)
# df['price'] = df['price'].astype(int)
# df.info()
# df_ = df['title'].str.extract( r'.*«(.*), (.*)м², (.*)/(.*)эт.».*')
# df_.columns=['obj_type','area','floor','nfloors']

In [13]:
# df[ df['description'].str.match(r'.*часть.*') ]
# df[ df['area'].isnull() ]
# df[ df['floor'].isnull() ]
# df[ df['nfloors'].isnull() ]
# df.query('is_openspace')

In [14]:
# df.sample(3)

In [15]:
# df.query('is_studio')

In [16]:
# # df_[ df_['obj_type'].isnull() ] 
# студия
# своб. планировка
# апартаменты
# доля
# аукцион

In [17]:
# list(df['title'].str.replace(r',.*','').drop_duplicates())

# ['1-к. квартира',
#  '2-к. квартира',
#  '3-к. апартаменты',
#  '3-к. квартира',
#  'квартира-студия',
#  '1-к. апартаменты',
#  'своб. планировка',
#  'апартаменты-студия',
#  '2-к. апартаменты',
#  '4-к. квартира',
#  'доля в 1-к. квартире',
#  '7-к. квартира',
#  '5-к. квартира',
#  'доля в квартира-студии',
#  '6-к. квартира',
#  'аукцион: 3-к. квартира',
#  'доля в 2-к. квартире',
#  '9-к. квартира',
#  'доля в 3-к. квартире',
#  '4-к. апартаменты',
#  'аукцион: своб. планировка',
#  '5-к. апартаменты',
#  'доля в квартире-студии',
#  '8-к. квартира',
#  '',
#  'доля в 10 и более-к. квартире',
#  'доля в 4-к. квартире',
#  'доля в апартаменты-студии']